# TD Lesson 6

À partir des datasets population_communes.csv et surface_departements.csv, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

la somme des "Population municipale" du département
la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
la part (en pourcentage) de la population municipale par rapport à la population totale
la part (en pourcentage) de la population (municipale) du département au sein de sa région
la densité de la population (municipale) en nb d'habitants / km2
(Le dataset final devrait ressembler à result-exo-cc.csv)

In [23]:
import pandas as pd
import numpy as np

In [2]:
dfpop = pd.read_csv('population_communes.csv')
dfpop1 = pd.read_csv('population_communes.csv')
dfsurf = pd.read_csv('surface_departements.csv')

In [3]:
dfsurf.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [4]:
dfpop.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [5]:
dfpop = dfpop.drop(['Code canton','Code arrondissement','Nom de la commune', 'Code commune'],axis=1)\
.groupby(['Code région','Code département','variable'])['value'].sum().reset_index()

In [6]:
dfpop

,Code région,Code département,variable,value
0,1,971,Population municipale,394110
1,1,971,Population totale,400170
2,2,972,Population municipale,376480
3,2,972,Population totale,382294
4,3,973,Population municipale,269352
...,...,...,...,...
195,93,84,Population totale,570921
196,94,2A,Population municipale,154303
197,94,2A,Population totale,156958
198,94,2B,Population municipale,176152


In [7]:
dfpop = dfpop.pivot_table(values='value', columns='variable', index=['Code région','Code département'])

In [8]:
dfpop.head()

,variable,Population municipale,Population totale
Code région,Code département,,
1,971,394110,400170
2,972,376480,382294
3,973,269352,271829
4,974,852924,862814
11,75,2190327,2210875


In [9]:
dfpop['pct_pop_municipale'] = dfpop['Population municipale'] / dfpop['Population totale'] * 100

In [10]:
dfpop.head()

,variable,Population municipale,Population totale,pct_pop_municipale
Code région,Code département,,,
1,971,394110,400170,98.485644
2,972,376480,382294,98.479181
3,973,269352,271829,99.088765
4,974,852924,862814,98.853751
11,75,2190327,2210875,99.070594


In [14]:
dfpop1 = dfpop1.drop(['Code canton','Code arrondissement','Nom de la commune', 'Code commune'],axis=1)\
.groupby(['Code région','Code département','variable'])['value'].sum().reset_index()

In [15]:
dfpop1 = dfpop1.pivot_table(values='value', columns='variable', index=['Code région','Code département'])

In [24]:
dfpop1.groupby('Code région').transform(np.sum)['Population municipale'] #transform('sum')['Population municipale']

Code région  Code département
1            971                   394110
2            972                   376480
3            973                   269352
4            974                   852924
11           75                  12117132
                                   ...   
93           13                   5021928
             83                   5021928
             84                   5021928
94           2A                    330455
             2B                    330455
Name: Population municipale, Length: 100, dtype: int64

In [25]:
dfpop['part_region'] = dfpop['Population municipale'] / dfpop.groupby('Code région').transform(sum)['Population municipale'] * 100

In [26]:
dfpop.head()

,variable,Population municipale,Population totale,pct_pop_municipale,part_region
Code région,Code département,,,,
1,971,394110,400170,98.485644,100.000000
2,972,376480,382294,98.479181,100.000000
3,973,269352,271829,99.088765,100.000000
4,974,852924,862814,98.853751,100.000000
11,75,2190327,2210875,99.070594,18.076282


In [27]:
dfmerge = dfpop.merge(dfsurf,left_on=['Code département'], right_on=['code_insee'])

In [28]:
dfmerge

,Population municipale,Population totale,pct_pop_municipale,part_region,code_insee,nom,surf_km2
0,394110,400170,98.485644,100.000000,971,Guadeloupe,1634.0
1,376480,382294,98.479181,100.000000,972,Martinique,1089.0
2,269352,271829,99.088765,100.000000,973,Guyane,83543.0
3,852924,862814,98.853751,100.000000,974,La Réunion,2505.0
4,2190327,2210875,99.070594,18.076282,75,Paris,105.0
...,...,...,...,...,...,...,...
95,2019717,2047433,98.646305,40.217960,13,Bouches-du-Rhône,5247.0
96,1055821,1073201,98.380546,21.024216,83,Var,6030.0
97,559014,570921,97.914422,11.131462,84,Vaucluse,3577.0
98,154303,156958,98.308465,46.694104,2A,Corse-du-Sud,4017.0


In [29]:
dfmerge['densité'] = dfmerge['Population municipale'] / dfmerge['surf_km2']

In [30]:
dfmerge

,Population municipale,Population totale,pct_pop_municipale,part_region,code_insee,nom,surf_km2,densité
0,394110,400170,98.485644,100.000000,971,Guadeloupe,1634.0,241.193390
1,376480,382294,98.479181,100.000000,972,Martinique,1089.0,345.711662
2,269352,271829,99.088765,100.000000,973,Guyane,83543.0,3.224112
3,852924,862814,98.853751,100.000000,974,La Réunion,2505.0,340.488623
4,2190327,2210875,99.070594,18.076282,75,Paris,105.0,20860.257143
...,...,...,...,...,...,...,...,...
95,2019717,2047433,98.646305,40.217960,13,Bouches-du-Rhône,5247.0,384.927959
96,1055821,1073201,98.380546,21.024216,83,Var,6030.0,175.094693
97,559014,570921,97.914422,11.131462,84,Vaucluse,3577.0,156.280123
98,154303,156958,98.308465,46.694104,2A,Corse-du-Sud,4017.0,38.412497


In [33]:
dfmerge = dfmerge.rename(columns={'nom':'nom_dept','code_insee':'code_dept'})\
[['nom_dept','code_dept','Population municipale','Population totale',\
    'pct_pop_municipale','part_region','densité']]

In [34]:
dfmerge.head()

,nom_dept,code_dept,Population municipale,Population totale,pct_pop_municipale,part_region,densité
0,Guadeloupe,971,394110,400170,98.485644,100.000000,241.193390
1,Martinique,972,376480,382294,98.479181,100.000000,345.711662
2,Guyane,973,269352,271829,99.088765,100.000000,3.224112
3,La Réunion,974,852924,862814,98.853751,100.000000,340.488623
4,Paris,75,2190327,2210875,99.070594,18.076282,20860.257143


In [35]:
dfmerge.to_csv('result-exo-pop.csv', index=False)